In [4]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_csv('songs_with_attributes_and_lyrics.csv')  # ganti path sesuai lokasi file
features = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact Nearest Neighbor (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_scaled.shape[1])
p_hnsw.init_index(max_elements=X_scaled.shape[0], ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])
nlist = 100
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], nlist, faiss.METRIC_L2)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Contoh tampilkan top-5 neighbors dari item pertama
# -------------------------------
print("\nTop-5 neighbors for first song:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")

Exact NN done in 292.800 s
Annoy done in 27.246 s
HNSW done in 36.224 s
FAISS IVF done in 51.168 s

Top-5 neighbors for first song:
Exact NN: [     0 394553 764272 837727 749223]
Annoy:    [0, 764272, 833164, 676733, 523698]
HNSW:     [     0 394553 764272 837727 749223]
FAISS:    [     0 394553 764272 837727 749223]


Buat dan tuliskan analisa anda terhadap code diatas.

Kode diatas membandingkan 4 metode ANN pada dataset lagu spotify. tujuan dari kode ini yaitu membandingkan kecepatan dan akurasi dari metode Exat NN, annoy, hnsw, dan faiss ivf.

pada bagian pertama dilakukan preprocessing dengan fungsi StandardScaler pada fitur X

Pada metode pertama menggunakan ENN dengan matrix distance Euclidean, method ini membandingkan setiap titik ke titik lainnya, sehingga untuk dataset besar akan terlalu lama prosesnya, namun akurasi mencapai 100%.

pada metode kedua menggunakan Annoy, method ini membuat 10 tree untuk membagi ruang vector, hasil proses jauh lebih cepat menjadi 27,246 detik, namun bisa ada tetangga yang terlewat, hasil tetangga juga sedikit berbeda dengan ENN karena approximate.

pada metode ketiga menggunakan HNSW, metode ini menggunakan graph-based indexing, setiap vector dihubungkan ke vector lain yang dekat sehingga seperti membentuk jaring jarring, hasil dari method ini sama persis dengan ENN dengan akurasi 100%, untuk kecepatan juga jauh lebih baik menjadi 36,224 detik dari ENN.

pada method keempat menggunakan FAISS IVF, method ini membagi data menjadi beberapa cluster(nlist), saat query hanya cluster terdekat(nprobe) cluster yang diperiksa, hasil tetangga juga sama persis dengan ENN dan kecepatannya juga jauh lebih baik menjadi 51,168 detik dari ENN dengan akurasi 100%

kesimpulannya, hasil dari ENN paling akurat, namun Waktu proses nya sangat lama untuk dataset besar sehingga tidak cocok untuk kondisi realtime. annoy menghasilkan kecepatan yang paling cepat, cocok untuk rekomendasi lagu seperti spotify namun akurasi yang kurang sempurna. HNSW merupakan metode ayng seimbang, hasil sama persis dengan ENN pada konfigurasi yang tepat, sehingga cocok untuk sistem besar dengan kebutuhan akurasi tinggi. Terakhir FAISS IVF lebih cocok digunakan untuk dataset yang sangat besar sehingga cocok digunakan untuk skala industri, contohnya META.
